In [1]:
import os
import shutil
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
BASE_DIR = '..'
RANDOM_SEED = 7 # for reproducibility
COUNTRIES_DIR = os.path.join(BASE_DIR, 'data', 'countries')
PROCESSED_DIR = os.path.join(BASE_DIR, 'data', 'processed')

# these relate to training the CNN to predict nightlights
CNN_TRAIN_IMAGE_DIR = os.path.join(BASE_DIR, 'data', 'cnn_images')
CNN_SAVE_DIR = os.path.join(BASE_DIR, 'models')


In [3]:
os.makedirs(CNN_TRAIN_IMAGE_DIR, exist_ok=True)
os.makedirs(CNN_SAVE_DIR, exist_ok=True)

# Preprocess
After doing this once, you can skip to the training if the script broke

In [4]:
df_download = pd.read_csv(os.path.join(PROCESSED_DIR, 'image_download_locs.csv'))
downloaded = os.listdir(os.path.join(COUNTRIES_DIR, 'malawi_2016', 'images')) + \
            os.listdir(os.path.join(COUNTRIES_DIR, 'ethiopia_2015', 'images')) + \
            os.listdir(os.path.join(COUNTRIES_DIR, 'nigeria_2015', 'images'))

print(f"actually downloaded: {len(downloaded)}, expected: {len(df_download)}")

actually downloaded: 32822, expected: 33360


In [5]:
df_download.head()

,image_name,image_lat,image_lon,cluster_lat,cluster_lon,cons_pc,nightlights,country,nightlights_bin
0,-17.09515_35.17229723579403_-17.09515_35.21721...,-17.095150,35.172297,-17.09515,35.217213,1.423239,0.025206,mw,0
1,-17.08017807859801_35.17229723579403_-17.09515...,-17.080178,35.172297,-17.09515,35.217213,1.423239,0.025206,mw,0
2,-17.125093842803985_35.18726915719602_-17.0951...,-17.125094,35.187269,-17.09515,35.217213,1.423239,0.025206,mw,0
3,-17.140065764205975_35.20224107859801_-17.0951...,-17.140066,35.202241,-17.09515,35.217213,1.423239,0.025206,mw,0
4,-17.065206157196016_35.20224107859801_-17.0951...,-17.065206,35.202241,-17.09515,35.217213,1.423239,0.025206,mw,0


In [6]:
df_download['row'] = np.arange(len(df_download))

In [7]:
downloaded =  list(set(downloaded).intersection(set(df_download['image_name'])))
idx_not_download = df_download.set_index('image_name').drop(downloaded).reset_index()#.values.tolist()
idx_not_download = list(idx_not_download['image_name'])
idx_not_download = list(set(idx_not_download).intersection(set(df_download['image_name'])))
df_download.set_index('image_name').drop(idx_not_download).reset_index()

,image_name,image_lat,image_lon,cluster_lat,cluster_lon,cons_pc,nightlights,country,nightlights_bin,row
0,-17.09515_35.17229723579403_-17.09515_35.21721...,-17.095150,35.172297,-17.095150,35.217213,1.423239,0.025206,mw,0,0
1,-17.08017807859801_35.17229723579403_-17.09515...,-17.080178,35.172297,-17.095150,35.217213,1.423239,0.025206,mw,0,1
2,-17.125093842803985_35.18726915719602_-17.0951...,-17.125094,35.187269,-17.095150,35.217213,1.423239,0.025206,mw,0,2
3,-17.140065764205975_35.20224107859801_-17.0951...,-17.140066,35.202241,-17.095150,35.217213,1.423239,0.025206,mw,0,3
4,-17.065206157196016_35.20224107859801_-17.0951...,-17.065206,35.202241,-17.095150,35.217213,1.423239,0.025206,mw,0,4
...,...,...,...,...,...,...,...,...,...,...
31884,13.729219383801992_5.587496745063984_13.714247...,13.729219,5.587497,13.714247,5.557553,1.667379,0.000000,ng,0,33355
31885,13.744191305203984_5.587496745063984_13.714247...,13.744191,5.587497,13.714247,5.557553,1.667379,0.000000,ng,0,33356
31886,13.759163226605976_5.587496745063984_13.714247...,13.759163,5.587497,13.714247,5.557553,1.667379,0.000000,ng,0,33357
31887,13.669331698194023_5.602468666465976_13.714247...,13.669332,5.602469,13.714247,5.557553,1.667379,0.000000,ng,0,33358


In [8]:
df_download.drop('row', axis=1, inplace=True)

In [9]:
# the distribution
(df_download['nightlights_bin']==0).mean(), (df_download['nightlights_bin']==1).mean(), (df_download['nightlights_bin']==2).mean()


(0.5091726618705036, 0.32146282973621104, 0.16936450839328537)

Split images into train/valid.
Each cluster will contribute 80% of images for training, and 20% for validation.

In [10]:
df_download.reset_index(drop=True, inplace=True)

In [11]:
df_download.head()

,image_name,image_lat,image_lon,cluster_lat,cluster_lon,cons_pc,nightlights,country,nightlights_bin
0,-17.09515_35.17229723579403_-17.09515_35.21721...,-17.095150,35.172297,-17.09515,35.217213,1.423239,0.025206,mw,0
1,-17.08017807859801_35.17229723579403_-17.09515...,-17.080178,35.172297,-17.09515,35.217213,1.423239,0.025206,mw,0
2,-17.125093842803985_35.18726915719602_-17.0951...,-17.125094,35.187269,-17.09515,35.217213,1.423239,0.025206,mw,0
3,-17.140065764205975_35.20224107859801_-17.0951...,-17.140066,35.202241,-17.09515,35.217213,1.423239,0.025206,mw,0
4,-17.065206157196016_35.20224107859801_-17.0951...,-17.065206,35.202241,-17.09515,35.217213,1.423239,0.025206,mw,0


In [12]:
df_download['is_train'] = True

                                              image_name  image_lat  \
0      -17.09515_35.17229723579403_-17.09515_35.21721... -17.095150   
1      -17.08017807859801_35.17229723579403_-17.09515... -17.080178   
2      -17.125093842803985_35.18726915719602_-17.0951... -17.125094   
3      -17.140065764205975_35.20224107859801_-17.0951... -17.140066   
4      -17.065206157196016_35.20224107859801_-17.0951... -17.065206   
...                                                  ...        ...   
33355  13.729219383801992_5.587496745063984_13.714247...  13.729219   
33356  13.744191305203984_5.587496745063984_13.714247...  13.744191   
33357  13.759163226605976_5.587496745063984_13.714247...  13.759163   
33358  13.669331698194023_5.602468666465976_13.714247...  13.669332   
33359  13.759163226605976_5.602468666465976_13.714247...  13.759163   

       image_lon  cluster_lat  cluster_lon   cons_pc  nightlights country  \
0      35.172297   -17.095150    35.217213  1.423239     0.025206     

In [13]:
np.random.seed(RANDOM_SEED)
groups = df_download.groupby(['cluster_lat', 'cluster_lon'])
for _, g in groups:
    n_ims = len(g)
    n_train = int(0.8 * n_ims)
    n_valid = n_ims - n_train
    valid_choices = np.random.choice(np.arange(n_ims), replace=False, size=n_valid).tolist()
    current_index = g.index
    idx_valid = current_index[valid_choices]
    df_download['is_train'].loc[idx_valid] = False

/home/sandesh/.local/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [14]:
df_download['is_train'].mean()

0.7721223021582734

In [15]:
# save this new dataframe
df_download.to_csv(os.path.join(PROCESSED_DIR, 'image_download_actual.csv'), index=False)

In [16]:
os.makedirs(os.path.join(CNN_TRAIN_IMAGE_DIR, 'train'), exist_ok=True)
os.makedirs(os.path.join(CNN_TRAIN_IMAGE_DIR, 'valid'), exist_ok=True)

labels = ['0', '1', '2']
for l in labels:
    os.makedirs(os.path.join(CNN_TRAIN_IMAGE_DIR, 'train', l), exist_ok=True)
    os.makedirs(os.path.join(CNN_TRAIN_IMAGE_DIR, 'valid', l), exist_ok=True)

In [17]:
t = df_download[df_download['is_train']]
v = df_download[~df_download['is_train']]

In [18]:
len(t), len(v)
print(t)

                                              image_name  image_lat  \
0      -17.09515_35.17229723579403_-17.09515_35.21721... -17.095150   
1      -17.08017807859801_35.17229723579403_-17.09515... -17.080178   
3      -17.140065764205975_35.20224107859801_-17.0951... -17.140066   
4      -17.065206157196016_35.20224107859801_-17.0951... -17.065206   
5      -17.11012192140199_35.217213_-17.09515_35.2172... -17.110122   
...                                                  ...        ...   
33353  13.669331698194023_5.587496745063984_13.714247...  13.669332   
33354  13.684303619596015_5.587496745063984_13.714247...  13.684304   
33355  13.729219383801992_5.587496745063984_13.714247...  13.729219   
33358  13.669331698194023_5.602468666465976_13.714247...  13.669332   
33359  13.759163226605976_5.602468666465976_13.714247...  13.759163   

       image_lon  cluster_lat  cluster_lon   cons_pc  nightlights country  \
0      35.172297   -17.095150    35.217213  1.423239     0.025206     

In [19]:
# uses symlinking to save disk space
print('copying train images')
for im_name, nl, country in tqdm(zip(t['image_name'], t['nightlights_bin'], t['country']), total=len(t)):
    country_dir = None
    if country == 'mw':
        country_dir = 'malawi_2016'
    elif country == 'eth':
        country_dir = 'ethiopia_2015'
    elif country == 'ng':
        country_dir = 'nigeria_2015'
    else:
        print(f"no match for country {country}")
        raise ValueError()
    src = os.path.abspath(os.path.join(COUNTRIES_DIR, country_dir, 'images', im_name))
    dest = os.path.join(CNN_TRAIN_IMAGE_DIR, 'train', str(nl), im_name)
    if os.symlink(src, dest,target_is_directory = False) != None:
        print("error creating symlink")
        raise ValueError()

print('copying valid images')
for im_name, nl, country in tqdm(zip(v['image_name'], v['nightlights_bin'], v['country']), total=len(v)):
    country_dir = None
    if country == 'mw':
        country_dir = 'malawi_2016'
    elif country == 'eth':
        country_dir = 'ethiopia_2015'
    elif country == 'ng':
        country_dir = 'nigeria_2015'
    else:
        print(f"no match for country {country}")
        raise ValueError()
    src = os.path.abspath(os.path.join(COUNTRIES_DIR, country_dir, 'images', im_name))
    dest = os.path.join(CNN_TRAIN_IMAGE_DIR, 'valid', str(nl), im_name)
    if os.symlink(src, dest,target_is_directory = False) != None:
        print("error creating symlink")
        raise ValueError()

copying train images


FileExistsError: [Errno 17] File exists: '/home/sandesh/Documents/Minor Project/predicting-poverty-replication/data/countries/malawi_2016/images/-17.09515_35.17229723579403_-17.09515_35.217213.png' -> '/home/sandesh/Documents/Minor Project/predicting-poverty-replication/data/cnn_images/train/0/-17.09515_35.17229723579403_-17.09515_35.217213.png'

In [20]:
# shows count distribution in train folder, make sure this matches above
counts = []
for l in ['0', '1', '2']:
    counts.append(len(os.listdir(os.path.join(CNN_TRAIN_IMAGE_DIR, 'train', l))))
print(counts)
print([c/sum(counts) for c in counts])
print(sum(counts))

[12653, 7973, 3996]
[0.5138900170579157, 0.32381609942328, 0.16229388351880433]
24622


In [21]:
# shows count distribution in valid folder
counts = []
for l in ['0', '1', '2']:
    counts.append(len(os.listdir(os.path.join(CNN_TRAIN_IMAGE_DIR, 'valid', l))))
print(counts)
print([c/sum(counts) for c in counts])
print(sum(counts))

[3889, 2380, 998]
[0.5351589376634099, 0.32750791248107886, 0.1373331498555112]
7267


# Train Model
Heavily adapted from the PyTorch CNN training tutorial.

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [23]:
# Top level data directory.

data_dir = CNN_TRAIN_IMAGE_DIR

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "vgg"

# Number of classes in the dataset
num_classes = 3

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for, first 10 will be training the new layers, last 10 the whole model
num_epochs = 20

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

In [24]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = models.vgg11_bn(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.classifier[6].in_features
    model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
    input_size = 224
    return model_ft, input_size

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [25]:
# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
model_ft

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke

In [26]:
# you can modify the classifier part of the model by doing this
# model_ft.classifier = nn.Sequential(
#     nn.Linear(in_features=25088, out_features=4096, bias=True),
#     nn.ReLU(inplace=True),
#     nn.Dropout(p=0.5),
#     nn.Linear(in_features=4096, out_features=256, bias=True),
#     nn.ReLU(inplace=True),
#     nn.Dropout(p=0.5),
#     nn.Linear(in_features=256, out_features=3, bias=True),
# )

In [27]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
print("Initializing Datasets and Dataloaders...")
# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'valid']}
print(image_datasets)
print(os.path.join(data_dir, 'train'))
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'valid']}
print(dataloaders_dict['train'])
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device:', device)

Initializing Datasets and Dataloaders...
{'train': Dataset ImageFolder
    Number of datapoints: 24622
    Root location: /home/sandesh/Documents/Minor Project/predicting-poverty-replication/data/cnn_images/train
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           ), 'valid': Dataset ImageFolder
    Number of datapoints: 7267
    Root location: /home/sandesh/Documents/Minor Project/predicting-poverty-replication/data/cnn_images/valid
    StandardTransform
Transform: Compose(
               Resize(size=224, interpolation=PIL.Image.BILINEAR)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )}
/home/sandesh/Do

In [28]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=1e-4, momentum=0.9)

Params to learn:
	 classifier.6.weight
	 classifier.6.bias


In [29]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        if epoch > 10:
            # fine tune whole model
            for param in model_ft.parameters():
                param.requires_grad = True
            optimizer = optim.SGD(model_ft.parameters(), lr=1e-4, momentum=0.9)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            print(dataloaders[phase])
            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'valid':
                val_acc_history.append(epoch_acc)
                
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [30]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs)

Epoch 0/19
----------



train Loss: 0.8202 Acc: 0.6181



valid Loss: 0.7171 Acc: 0.6792

Epoch 1/19
----------



train Loss: 0.7817 Acc: 0.6419



valid Loss: 0.7046 Acc: 0.6886

Epoch 2/19
----------


...


valid Loss: 0.5478 Acc: 0.7683

Epoch 18/19
----------



train Loss: 0.6057 Acc: 0.7343



valid Loss: 0.5543 Acc: 0.7651

Epoch 19/19
----------



train Loss: 0.5974 Acc: 0.7383



valid Loss: 0.5213 Acc: 0.7798

Training complete in 2308m 22s
Best val Acc: 0.779827


In [31]:
path = os.path.join(CNN_SAVE_DIR, 'trained_model.pt')
assert not os.path.isfile(path), print('A model is already saved at this location')
print(f'Saving model to {path}')
torch.save(model_ft, path)

Saving model to /home/sandesh/Documents/Minor Project/predicting-poverty-replication/models/trained_model.pt


In [32]:
# you can run below if you want to see the final accuracy on nightlights over the train set
model_ft.eval()   # Set model to evaluate mode

criterion = nn.CrossEntropyLoss()
running_loss = 0.0
running_corrects = 0
total = 0

# Iterate over data.
for inputs, labels in tqdm(dataloaders_dict['train']):
    inputs = inputs.to(device)
    labels = labels.to(device)

    # forward
    # track history if only in train
    with torch.set_grad_enabled(False):
        outputs = model_ft(inputs)
        loss = criterion(outputs, labels)

        _, preds = torch.max(outputs, 1)

    # statistics
    running_loss += loss.item() * inputs.size(0)
    running_corrects += torch.sum(preds == labels.data)
    
    total += len(preds)
        
print(running_corrects.double()/total)


tensor(0.7721, dtype=torch.float64)
